# Задача:
* от заказчика передан был пример выгрузки из канала https://disk.yandex.ru/d/NcyC_Xp0EyE8ow
Что нужно сделать:

* распарсить json и положить в структуру pandas DataFrame. Пример того, что ожидается https://drive.google.com/file/d/1Ry4zMi91-qqUVLds2OVCmmG95kH8gllt/view?usp=sharing  

каждый участник создает свою ветку и пушит код парсинга

In [1]:
import json
import pandas as pd
import os

In [2]:
# робота с Google Диском
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Путь к json ввожу вручную
%cd /content/drive/My Drive/Colab Notebooks/Projects/Master_Ya
path_json = os.getcwd() + '/down/маркетинг/result.json'
path_json

/content/drive/My Drive/Colab Notebooks/Projects/Master_Ya


'/content/drive/My Drive/Colab Notebooks/Projects/Master_Ya/down/маркетинг/result.json'

In [4]:
# Загружаем JSON-файл
with open(path_json, 'r', encoding='utf-8') as file:
    chat_data = json.load(file)

In [5]:
# Извлекаем сообщения, а также название чата(в JSON-файл один чат) и chat_id
messages = chat_data.get('messages', [])
chat_name = chat_data.get('name')
chat_id = chat_data.get('id')

In [6]:
# Парсим текстовые сообщения
parsed_messages = []
for message in messages:
    # отсекаем служебные сообщения  type: service
    if message.get('type') == 'message':
        text = message['text']
        parsed_messages.append({
            'message_id': message['id'],
            'date': message['date'],
            'chat_name': chat_name,
            'chat_id': chat_id,
            'sender_id': message.get('from_id'),
            'from': message.get('from', 'Unknown'),
            'text': text
        })

In [7]:
# Создаем датафрейм
df = pd.DataFrame(parsed_messages)

In [8]:
df

,message_id,date,chat_name,chat_id,sender_id,from,text
0,42471,2025-02-01T07:35:54,💬 Marketing Practicum Chat,1782474569,user1272493086,Pasha Sannikov,"[Доброе утро, люди! Это ОК, в РСЯ алгоритмы вы..."
1,42472,2025-02-01T07:40:03,💬 Marketing Practicum Chat,1782474569,user1272493086,Pasha Sannikov,"А вот регионы, в которых живут ITшники с интер..."
2,42473,2025-02-01T07:48:53,💬 Marketing Practicum Chat,1782474569,user1272493086,Pasha Sannikov,"Если смотреть по интересам, то кажется, что ау..."
3,42474,2025-02-01T07:50:31,💬 Marketing Practicum Chat,1782474569,user1272493086,Pasha Sannikov,А путь пользователя почему-то выглядит так. Ко...
4,42475,2025-02-01T08:12:42,💬 Marketing Practicum Chat,1782474569,user1164775873,Данилов | Маркетолог,Опять он
...,...,...,...,...,...,...,...
3675,46400,2025-02-27T20:11:04,💬 Marketing Practicum Chat,1782474569,user1829741608,Алексей Ермолов,"Напишу им, спасибо."
3676,46401,2025-02-27T20:23:08,💬 Marketing Practicum Chat,1782474569,user1912824918,S,"Старые профили не удаляются, номер на который ..."
3677,46402,2025-02-27T20:24:44,💬 Marketing Practicum Chat,1782474569,user1912824918,S,Пишите оператору (позовите пожалуйста оператор...
3678,46403,2025-02-27T20:29:44,💬 Marketing Practicum Chat,1782474569,user1829741608,Алексей Ермолов,Отлично! Благодарю.


## Попробывал анализировать данные

In [9]:
# Топ 10 активных участников
message_counts = df['from'].value_counts()
print(message_counts.head(10))

from
Андрей Дамаскин         614
Олег                    563
Pasha Sannikov          455
Данилов | Маркетолог    405
Алексей Ермолов         315
Ed                      281
Екатерина               160
Андрей                  131
Дмитрий                  85
Марина                   66
Name: count, dtype: int64


In [10]:
# Топ 10 дней активноти
df['date'] = pd.to_datetime(df['date'])
df['day'] = df['date'].dt.date
active_days = df['day'].value_counts()
print(active_days.head(10))

day
2025-02-05    387
2025-02-06    301
2025-02-19    229
2025-02-25    221
2025-02-10    204
2025-02-03    192
2025-02-21    180
2025-02-18    168
2025-02-01    160
2025-02-12    157
Name: count, dtype: int64
